In [15]:
import copy, numpy as np

data = open('HP1.txt','r', encoding="utf8").read();
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(data_size,", ",vocab_size)

431677 ,  79


In [16]:
# Dictionary of input chars & indices
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{'x': 0, 'a': 1, 'n': 2, 'M': 3, '!': 4, 'm': 5, 'N': 6, 'I': 7, 'q': 8, 'o': 9, '(': 10, ')': 11, 'J': 12, 'O': 13, 'U': 14, '7': 15, '3': 16, '9': 17, '\\': 18, 'k': 19, '\t': 20, '"': 21, '~': 22, 'H': 23, ':': 24, 'C': 25, '.': 26, 'E': 27, 'y': 28, 'f': 29, 'p': 30, 'z': 31, '1': 32, 'j': 33, 'V': 34, '-': 35, 'i': 36, 'Z': 37, ',': 38, '2': 39, 'l': 40, 'L': 41, '5': 42, 'Y': 43, 'S': 44, 'w': 45, '\n': 46, 'B': 47, '?': 48, 'Q': 49, 'G': 50, 'h': 51, "'": 52, 'K': 53, 'g': 54, '8': 55, 'R': 56, '6': 57, 'A': 58, ';': 59, 'F': 60, 'W': 61, '4': 62, '0': 63, '*': 64, 'r': 65, 'D': 66, 'X': 67, 'c': 68, 's': 69, 'T': 70, 'u': 71, 'P': 72, 'e': 73, 'd': 74, 'b': 75, 't': 76, ' ': 77, 'v': 78}
{0: 'x', 1: 'a', 2: 'n', 3: 'M', 4: '!', 5: 'm', 6: 'N', 7: 'I', 8: 'q', 9: 'o', 10: '(', 11: ')', 12: 'J', 13: 'O', 14: 'U', 15: '7', 16: '3', 17: '9', 18: '\\', 19: 'k', 20: '\t', 21: '"', 22: '~', 23: 'H', 24: ':', 25: 'C', 26: '.', 27: 'E', 28: 'y', 29: 'f', 30: 'p', 31: 'z', 32: '1', 33: '

In [17]:
# demo of onehot encoding
vector_for_char_a = np.zeros((vocab_size,1))
vector_for_char_a[char_to_ix['a']] = 1
print(vector_for_char_a.ravel())

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


In [18]:
# Hyperparams
hidden_size = 100
seq_length = 25
learning_rate = 1e-1
# the lower the learning rate, the quicker the network abandons old belief for new input
# e.g. train images on dogs, give a cat, low learning rate will consider cat is anormally rather than dog

# Model params
Wxh1 = np.random.randn(hidden_size,  vocab_size)* 0.01 # input to hidden (input is onehot encoded)
Bxh1 = np.zeros((hidden_size,1))

# Hiddens
Wh1h2 = np.random.randn(hidden_size, hidden_size)* 0.01 # hidden1 to hidden 2
Wh1h1 = np.random.randn(hidden_size, hidden_size)* 0.01 # recurrent hidden .
Wh2h2 = np.random.randn(hidden_size, hidden_size)* 0.01 # recurrent hidden .

Bh1h2 = np.zeros((hidden_size,1))

# last hidden to Output 
Wh2y = np.random.randn(vocab_size,  hidden_size)* 0.01 # hidden to output(decode the output)
Bh2y = np.zeros((vocab_size,1))


In [19]:
# softmax helper
def softmax(seq):
    return np.exp(seq)/ np.sum(np.exp(seq))

def softmax_array(two_D_seq,t):
    return np.exp(two_D_seq[t])/ np.sum(np.exp(two_D_seq[t]))

In [20]:
# Loss function - training
def lossFunction(inputs, targets, prev_hidden1, prev_hidden2):
    # p is softmax probability
    xs, h1s, h2s, ys, ps = {},{},{},{},{};
    
    h1s[-1] = copy.deepcopy(prev_hidden1)
    h2s[-1] = copy.deepcopy(prev_hidden2)
    loss = 0;
    
    # Fwd pass    
    for t in range(len(inputs)):
        # One hot encoding for the input char using our dictionary
        xs[t] = np.zeros((vocab_size,1));
        xs[t][inputs[t]] = 1; 
        
        h1s[t] = np.tanh(np.dot(Wxh1,xs[t]) + np.dot(Wh1h1,h1s[t-1]) + Bxh1);
        h2s[t] = np.tanh(np.dot(Wh1h2,h1s[t]) + np.dot(Wh2h2,h2s[t-1]) + Bh1h2);
        
        ys[t] = np.dot(Wh2y,h2s[t]) + Bh2y;
        ps[t] = softmax(ys[t]);
        char_idx = targets[t]
        loss += -np.log(ps[t][char_idx,0]) 
        # ps[t][targets[t]] is the prob. node corrs. to. t_th char in the label array

        
    # Gradient value holders
    dWxh1, dWh1h2, dWh2y = np.zeros_like(Wxh1),np.zeros_like(Wh1h2),np.zeros_like(Wh2y)
    dWh1h1,dWh2h2        = np.zeros_like(Wh1h1),np.zeros_like(Wh2h2)
    dBxh1, dBh1h2, dBh2y = np.zeros_like(Bxh1),np.zeros_like(Bh1h2),np.zeros_like(Bh2y)
    dh1next = np.zeros_like(h1s[0])
    dh2next = np.zeros_like(h2s[0])
    
    # Bwd pass
    for t in reversed(range(len(inputs))):
        
        # prob. p to output y
        dy = copy.deepcopy(ps[t])
        dy[targets[t]] -= 1 # this is how we calculate loss using onehot encoding
        
        # y to h2 
        dWh2y += np.dot(dy, h2s[t].T);        
        dBh2y += dy # derivative w.r.t bias is 1        
        dh2 = np.dot(Wh2y.T,dy) + dh2next # back prop the error from y into h2
                
        # h2 to h1 and h2_prev
        dh2raw = (1-h2s[t]*h2s[t])*dh2 # back prop thru tanh
        dBh1h2 += dh2raw  # derivative of Wx+b w.r.t b is 1; d_loss/d_b = d_loss/d_H * d_H/d_b
        dWh1h2 += np.dot(dh2raw, h1s[t].T) # derivative of Wx+b w.r.t W is x
        dWh2h2 += np.dot(dh2raw, h2s[t-1].T)
        dh2next = np.dot(Wh2h2.T, dh2raw)
        
        dh1 = np.dot(Wh1h2.T,dh2) + dh1next # back prop the error from y into h2
        
        # h1 to x and h1_prev
        dh1raw = (1-h1s[t]*h1s[t])*dh1 # back prop thru tanh        
        dBxh1 += dh1raw  # derivative of Wx+b w.r.t b is 1; d_loss/d_b = d_loss/d_H * d_H/d_b
        dWxh1 += np.dot(dh1raw, xs[t].T) # derivative of Wx+b w.r.t W is x
        dWh1h1 += np.dot(dh1raw,h1s[t-1].T)
        dh1next = np.dot(Wh1h1.T, dh1raw)

    # Can be replaced using LSTM structure
    for dparam in [dWxh1, dWh1h1, dWh1h2, dWh2h2, dWh2y, dBxh1, dBh1h2, dBh2y]:
        np.clip(dparam,-5,5,out=dparam) # mitigate gradient vanish
        
        
    return loss,dWxh1,dWh1h2,dWh1h1,dWh2h2, dWh2y, dBxh1,dBh1h2, dBh2y, h1s[len(inputs)-1],h2s[len(inputs)-1];


 

In [22]:
# prediction
def sample(h1,h2,seed_ix,n):
    x = np.zeros((vocab_size,1))
    # one hot encode
    x[seed_ix] = 1;
    ixes = [] # empty sentence

    for t in range(n):

        h1 = np.tanh(np.dot(Wxh1 ,x ) + np.dot(Wh1h1,h1) + Bxh1 );
        h2 = np.tanh(np.dot(Wh1h2,h1) + np.dot(Wh2h2,h2) + Bh1h2);
        y = np.dot(Wh2y,h2) + Bh2y;
        p = softmax(y);

        # sample the output
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        
        # encode this output
        x = np.zeros((vocab_size,1))
        x[ix] = 1;

        ixes.append(ix)
        # if n > 1, it will predict more than 1 subsequent chars
        
    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print ("----\n %s \n----" % (txt,))
    
# test
h1prev = np.zeros((hidden_size,1)) # reset RNN memory  
h2prev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(h1prev,h2prev,char_to_ix['a'],200)

----
 F53dvW8pkVHVRH1u.U\J:aK:qrF'JN~wC )w0\fOd77,*!8KLoXfORU,RTtWa(U"CZyN.*a,)jQRg:VeC?TD3)pUxBB*.5anFG:sl0Zer5LAxGfZ40EXg~!P"14El.YUS )MjKBfhlLxXIIIa'?8s;U;PU6Ryon!vrTjY~2\DTE(N-VHxr0aKOWR
"ahS5GJ)qH~GpR( 
----


In [23]:
# Training using Adagrad (decreasing learning rate)
position = 0
inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
print(data[position:position+seq_length])
print("inputs",inputs)
targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 
print(data[position+1:position+seq_length+1])
print("targets",targets)



Harry Potter and the Sorc
inputs [23, 1, 65, 65, 28, 77, 72, 9, 76, 76, 73, 65, 77, 1, 2, 74, 77, 76, 51, 73, 77, 44, 9, 65, 68]
arry Potter and the Sorce
targets [1, 65, 65, 28, 77, 72, 9, 76, 76, 73, 65, 77, 1, 2, 74, 77, 76, 51, 73, 77, 44, 9, 65, 68, 73]


In [24]:
# Real training

n, position = 0,0

# for Adaptive Gradient descent
mWxh1 = np.zeros_like(Wxh1);
mWh1h1 = np.zeros_like(Wh1h1);
mWh1h2 = np.zeros_like(Wh1h2);
mWh2h2 = np.zeros_like(Wh2h2);
mWh2y = np.zeros_like(Wh2y);

mBxh1 = np.zeros_like(Bxh1);
mBh1h2 = np.zeros_like(Bh1h2);
mBh2y = np.zeros_like(Bh2y);

smooth_loss = -np.log(1.0/vocab_size)*seq_length;

epoch = 200*1000;
sample_length = 200;

while n<epoch:

    if(position+seq_length+1 >= len(data) or n == 0):
        
        h1prev = np.zeros((hidden_size,1))
        h2prev = np.zeros((hidden_size,1))
        position = 0;
        
    inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
    targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 
    
    loss,dWxh1,dWh1h2,dWh1h1,dWh2h2,dWh2y,dBxh1,dBh1h2,dBh2y,h1prev,h2prev = lossFunction(inputs,targets,h1prev, h2prev)  
    smooth_loss = smooth_loss*0.999+loss*0.001
    
    if(n%1000 == 0):
        print(n,"loss",smooth_loss)
        sample(h1prev,h2prev,inputs[0],sample_length);
        
    # update
    for param, dparam, mem in zip([Wxh1,Wh1h1,Wh1h2,Wh2h2,Wh2y,Bxh1,Bh1h2,Bh2y],
                                  [dWxh1,dWh1h1,dWh1h2,dWh2h2,dWh2y,dBxh1,dBh1h2,dBh2y],
                                  [mWxh1,mWh1h1,mWh1h2,mWh2h2,mWh2y,mBxh1,mBh1h2,mBh2y]):
        
        mem += dparam*dparam;
        param += -learning_rate * dparam *  1 / np.sqrt(mem +1e-8) # Adagrad
        
    position += seq_length;
    n += 1;
    
        

0 loss 109.236195616
----
 ~-3u7yvqhqA"0neWw)cLSPJx9x'UR-z!pYI0rLD\",cOyvQE
b0oPJVjzd3U0EORPEiMQdNer\jPx4?wrDY\6'r.dr*KX* *CAzr7o(0TYZ\at\R4Yuz'vRdzlggRrzHpnlvmX'4rhFr?a08OI0C-YqSOo\6\9v-WFn0g8FMDiK7PN*K(l8U\f~jil-.Q:PocE6dZm*E 
----
1000 loss 91.9845786678
----
 plge fa kfvnti.ne'oolrn-one tiula dh, tu. Doahhm cisZ
eedny alllflidors gnop ava ok xaef 9aai?ep g 'ohe ""cite os pnlne ini-okerssedo soeh. 
hegitehil lTn
itw?oes aiw
waa~ ghvdlthbs
fd aap e oaetie -n 
----
2000 loss 78.1893078601
----
 tf camed an ttes shurs watl haMttlawnmewsad aHes makro we, lonr irTtf'bor ts mna p-ie rarle sis nb
arseLe tons hudcte jfe" ap he on hepiilr. igt th bta fons woucwatts ono mit siilmBse, lecAinbeinweank 
----
3000 loss 71.380690793
----
 r kakin mas iwh che ot nalt fanh
onenrouyos ih ocaol ans. d thiog thed bamk ketung, id!acgins anr ire yretotnhe karle tonk Ik..
Hif ch ancwhe Yonet thetmewrortudlhoor scsimpinsornneheled ppet ov! rent 
----
4000 loss 68.4358421357
----
 meind wtany het'.gos af.
S

KeyboardInterrupt: 

In [43]:
print(n)

100000
